In [1]:
import gudhi as gd
from gudhi import representations
import numpy as np
import numpy.random
import matplotlib.pyplot as plt
import pingouin as pg
import scipy.stats as st
from collections import defaultdict
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import paired_distances
from tqdm.notebook import tqdm
import scipy.spatial.distance as spdists
from scipy.stats import ks_2samp

#from curlyBrace import curlyBrace
%matplotlib widget

c:\Users\Niklas\anaconda3\lib\site-packages\outdated\utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.0, the latest is 0.2.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
c:\Users\Niklas\anaconda3\lib\site-packages\outdated\utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.3, the latest is 0.5.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [2]:
def sampled_ecc_alpha(pointcloud, n_samples, max_filt_val):
    ac = gd.AlphaComplex(points = pointcloud)
    st = ac.create_simplex_tree()
    filt_values = np.linspace(0,max_filt_val,n_samples)
    ecc_values = np.zeros_like(filt_values)
    simplices = list(st.get_filtration())
    cur_val = 0
    i=0
    for (j,t) in enumerate(filt_values):
        while (i<len(simplices)) and (simplices[i][1]<=t):
            cur_val=cur_val +(-1)**(len(simplices[i][0])-1)
            i = i+1
        #print(t, cur_val)
        ecc_values[j] = cur_val
        
    return ecc_values

In [3]:
def get_ecc(data, n_samples, max_filt_val):
    eccs = []
    for dat in tqdm(data):
        ec = sampled_ecc_alpha(dat, n_samples, max_filt_val)
        eccs.append(ec)
    return eccs

In [4]:
def ecc_distance(samples1, samples2, n_samples, max_filt_val):
    ECCs1 = get_ecc(samples1, n_samples = n_samples, max_filt_val = max_filt_val)
    ECCs2 = get_ecc(samples2, n_samples = n_samples, max_filt_val = max_filt_val)
    #dists = paired_distances(ECCs1,ECCs2, n_jobs=-1, metric = spdists.chebyshev)
    dists = np.maximum(ECCs1-ECCs2)+np.maximum(ECCs2-ECCs1)
    return dists
    #dists = dists *(max_filt_val/(n_samples-1))
    #return dists.flatten()

In [5]:
def gen_rv(rv, N, n_samples):
    return [np.array([rv[0].rvs(N), rv[1].rvs(N).tolist(), rv[2].rvs(N).tolist()]).T for i in range(n_samples)]

In [6]:
rv_normal = st.norm()
rv_normal2_2 = st.norm(2,2)
rv_beta2_2 = st.beta(2, 2)
rv_beta5_5 = st.beta(5, 5)
rv_beta10_10 = st.beta(10, 10)
rv_laplace = st.laplace()
rv_uniform = st.uniform()
rv_t3 = st.t(df=3)
rv_t5 = st.t(df=5)
rv_t7 = st.t(df=7)
rv_t10 = st.t(df=10)
rv_gamma10_1 = st.gamma(10,1)
rv_gamma20_1 = st.gamma(20,1)
rv_gamma4_5 = st.gamma(4,5)
rv_gamma5_1= st.gamma(5,1)
rv_chisq4 = st.chi2(df=4)
rv_chisq10 = st.chi2(df=10)
rv_lorentz = st.cauchy()
rv_logistic = st.logistic()
rv_arcsine = st.arcsine()
rv_argus = st.argus(chi=1)
rv_cosine = st.cosine()

In [7]:
n=200
n_loops = 200
d = 3

rvs= [[rv_uniform, rv_uniform, rv_uniform],
      [rv_beta2_2, rv_beta2_2, rv_beta2_2],
      [rv_beta5_5, rv_beta5_5,rv_beta5_5],
      [rv_beta10_10, rv_beta10_10, rv_beta10_10],
      [rv_argus, rv_argus, rv_argus],
      [rv_arcsine, rv_arcsine, rv_arcsine],
      [rv_uniform, rv_beta10_10, rv_beta10_10],
      [rv_uniform, rv_uniform, rv_beta10_10]]

rv_names=["UxUxU",
          "B22xB22XB22",
          "B55xB55xB55",
          "B1010xB1010xB1010",
          "ArgusxArgusxArgus",
          "asinxasinxasin",
          "UxB1010xB1010",
          "UxUxB1010"]

In [8]:
samples = [(n**(1/3)) * np.array(gen_rv(rv,n,n_loops)) for rv in tqdm(rvs)]
samples2 = [(n**(1/3)) * np.array(gen_rv(rv,n,n_loops)) for rv in tqdm(rvs)]

In [32]:
ecc_dists = np.zeros((len(rvs),n_loops))
eccs1 = np.array([get_ecc(s, 101,0.5) for s in samples])
eccs2 = np.array([get_ecc(s, 101,0.5) for s in samples2])

#eccs_mean = np.mean(eccs1, axis =1)


In [34]:

for i in tqdm(range(0,len(rvs))):
    #ecc_dists[i] = paired_distances(eccs1[i], eccs2[i], n_jobs = -1, metric = spdists.chebyshev)
    index_plus = np.argmax(eccs1[i]-eccs2[i],axis=1)
    index_minus = np.argmax(eccs2[i]-eccs1[i],axis=1)

    ecc_dists[i] = (np.log(eccs1[i][index_plus]/eccs2[i][index_plus]))+(np.log(eccs2[i][index_minus]-eccs1[i][index_minus]))

c:\Users\Niklas\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  
c:\Users\Niklas\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  
c:\Users\Niklas\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
c:\Users\Niklas\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in log
  
c:\Users\Niklas\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in add
  


ValueError: could not broadcast input array from shape (200,101) into shape (200,)

In [35]:
eccs1[:,index_plus]/eccs2[:,index_plus]
#eccs1[0][0]

c:\Users\Niklas\anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.
c:\Users\Niklas\anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([[[  1.        ,   0.985     ,   0.98994975, ...,   0.67346939,
           0.75555556,   0.80487805],
        [  1.        ,   1.        ,   0.99492386, ...,   1.02564103,
           1.        ,   0.84444444],
        [  1.        ,   1.01010101,   1.01020408, ...,   0.77272727,
           0.76744186,   0.76190476],
        ...,
        [  1.        ,   1.        ,   0.98984772, ...,   1.3125    ,
           1.17142857,   1.25714286],
        [  1.        ,   1.00502513,   1.01025641, ...,   0.55813953,
           0.65217391,   0.58333333],
        [  1.        ,   1.        ,   0.99      , ...,   1.13636364,
           1.09302326,   1.13636364]],

       [[  1.        ,   0.9798995 ,   1.        , ...,   1.44444444,
           1.22222222,   1.625     ],
        [  1.        ,   0.98994975,   1.02094241, ..., -18.        ,
          16.        , -15.        ],
        [  1.        ,   1.01010101,   1.02604167, ...,   1.5       ,
           1.28571429,   1.        ],
        ...,


In [16]:
f,ax = plt.subplots()
ax.hist(ecc_dists.T)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
pvals = np.zeros((len(rvs),len(rvs)))
for i in range(0,len(rvs)):
    for j in range(0,len(rvs)):
        pvals[i][j] = ks_2samp(ecc_dists[i], ecc_dists[j])[1]

plt.matshow(pvals)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
pvals = np.zeros((len(rvs),len(rvs)))
for i in range(0,len(rvs)):
    for j in range(0,len(rvs)):
        pvals[i][j] = ks_2samp(pairwise_distances(samples[i][0], n_jobs=-1).flatten(),pairwise_distances(samples2[j][0], n_jobs=-1).flatten())[0]

plt.matshow(pvals)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …